<a href="https://colab.research.google.com/github/rafayk7/DeepFunctionApproximator/blob/main/MainAlgo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.optim as optim
import torch.nn.functional as F
import random
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from PIL import Image

import numpy as np

import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler

In [ ]:
from google.colab import drive
import pickle
import os
# Mounting Google Drive
drive.mount('/content/drive')

data_dir = '/content/drive/MyDrive/APS360 - Project/Data'

with open(os.path.join(data_dir, 'dataset_info.pkl'), 'rb') as f:
    data = pickle.load(f)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Shuffle by converting to list first
dataset_list = list(data.items())
random.shuffle(dataset_list)
shuffled_dataset = dict(dataset_list)

# Sanity check
# Expect 8004 images
print(len(shuffled_dataset))
print(shuffled_dataset)

8000
{'polynomial_degree_2_42.png': [-0.1, 0.0, -1.4, 0.0], 'polynomial_degree_0_1491.png': [-0.1, 0.0, 0.0, 0.0], 'polynomial_degree_0_777.png': [1.6, 0.0, 0.0, 0.0], 'polynomial_degree_0_1114.png': [2.0, 0.0, 0.0, 0.0], 'polynomial_degree_0_582.png': [-0.0, 0.0, 0.0, 0.0], 'polynomial_degree_3_1184.png': [-0.4, 0.5, -1.0, 0.2], 'polynomial_degree_3_1247.png': [-0.8, -0.1, 0.5, -2.3], 'polynomial_degree_2_301.png': [-0.8, -0.5, -1.6, 0.0], 'polynomial_degree_3_476.png': [1.1, -0.9, 0.1, 0.0], 'polynomial_degree_1_961.png': [1.4, 0.2, 0.0, 0.0], 'polynomial_degree_2_1223.png': [0.2, -1.0, 0.2, 0.0], 'polynomial_degree_1_1136.png': [0.7, 1.3, 0.0, 0.0], 'polynomial_degree_0_1377.png': [0.5, 0.0, 0.0, 0.0], 'polynomial_degree_3_1476.png': [0.3, -2.5, 1.7, -1.4], 'polynomial_degree_2_1317.png': [0.7, 0.5, 1.0, 0.0], 'polynomial_degree_2_577.png': [-0.6, 0.0, 0.1, 0.0], 'polynomial_degree_1_376.png': [-0.0, -0.8, 0.0, 0.0], 'polynomial_degree_3_284.png': [1.5, 0.8, 1.6, 0.0], 'polynomial_d

In [ ]:
# Define the custom dataset class
class PolynomialDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.filenames = os.listdir(data_dir)
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        # Load image and label
        filename = self.filenames[idx]
        img = Image.open(os.path.join(self.data_dir, filename))
        label = torch.FloatTensor(shuffled_dataset[filename])

        # Apply transform if specified
        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
# Set up sub-directory paths and join to 
# main data directory
train_dir = os.path.join(data_dir, 'Train')
val_dir = os.path.join(data_dir, 'Val')
test_dir = os.path.join(data_dir, 'Test')

# Define the transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels = 1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Create data loaders for the training, validation, and testing datasets
train_dataset = PolynomialDataset(train_dir, transform=transform)
val_dataset = PolynomialDataset(val_dir, transform=transform)
test_dataset = PolynomialDataset(test_dir, transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
vgg19 = models.vgg19()

In [ ]:
MAX_DEGREES = 3

# Set this up as a regression problem
vgg19.classifier._modules['6'] = nn.Linear(4096, MAX_DEGREES+1)

# Don't re-train feature extractor
for param in vgg19.parameters():
   param.requires_grad = False

# Re-Train Regression Component
for param in vgg19.classifier.parameters():
   param.requires_grad = True

In [ ]:
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    return path

In [ ]:
def train(model, train_dataset, val_dataset, batch_size=128, num_epochs=20, learn_rate=0.001):
    torch.manual_seed(10)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learn_rate)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

    train_loss = np.zeros(num_epochs)
    val_loss = np.zeros(num_epochs)

    # training
    print ("Training Started...")
    n = 0
    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_train_err = 0.0
        total_images = 0
        for imgs, labels in iter(train_loader):
            padding = torch.autograd.Variable(torch.zeros(batch_size, 2, imgs.shape[2], imgs.shape[3]))
            imgs = torch.cat((imgs, padding), 1)
            print(imgs.shape)
            print(padding.shape)

            
            if torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()

            out = model(imgs)             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch
            n += 1

            train_loss = float(loss)

        for imgs, labels in iter(val_loader):
            padding = torch.autograd.Variable(torch.zeros(batch_size, 2, img.shape[2], img.shape[3]))
            imgs = torch.cat((imgs, padding), 1)
            
            if torch.cuda.is_available():
              imgs = imgs.cuda()
              labels = labels.cuda()

            out = model(imgs)             # forward pass
            loss = criterion(out, labels) # compute the total loss
            val_loss = float(loss)

        # track accuracy
        train_loss[epoch] = train_loss/batch_size
        val_loss[epoch] = val_loss/batch_size

        print(("Epoch {}: Train acc: {} |" + "Validation acc: {}").format(epoch, train_loss[epoch], val_loss[epoch]))

        model_path = get_model_name(model.name, batch_size, learn_rate, epoch)
        torch.save(model.state_dict(), model_path)
            
    epochs = np.arange(1, num_epochs + 1)
    
    return train_loss, val_loss, epochs


In [ ]:
train(vgg19, train_dataset, val_dataset, batch_size=256, num_epochs=20, learn_rate=0.1)

Training Started...


In [ ]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

for a,b in iter(train_loader):
  print(a.shape)
  break
  

torch.Size([128, 1, 224, 224])


In [ ]:
loss = nn.MSELoss()

In [ ]:
for img, label in train_dataset:
  padding = torch.autograd.Variable(torch.zeros(2, img.shape[1], img.shape[2]))
  img_pad = torch.cat((img, padding), 0)

  pred = vgg19(torch.unsqueeze(img_pad, dim=0))
  print(pred)
  nn.MSELoss()
  print(loss(pred,label))
  break

tensor([[ 0.0063,  0.0442, -0.0125, -0.0601]], grad_fn=<AddmmBackward0>)
tensor(0.1947, grad_fn=<MseLossBackward0>)


/usr/local/lib/python3.9/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
